In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization

# Data Science
import numpy as np
import pandas as pd


# Feature Engineering
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.metrics import r2_score

# Others
import os
from tqdm import tqdm
from shapely.geometry import Point, Polygon
from shapely import wkt
import geopandas as gpd

### Utility Functions: model fitting, data splitting, table merging ###

In [2]:
def get_data(uhi_data, used_columns, split_ratio=0.3):
    kept_data = uhi_data[used_columns]
    X = kept_data.drop(columns=['UHI Index']).values
    y = kept_data ['UHI Index'].values
    if split_ratio == 0.0:
        return X, None, y, None
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_ratio,random_state=123)
    return X_train, X_test, y_train, y_test

def get_perf(uhi_data, model, used_columns, split_ratio=0.3):
    X_train, X_test, y_train, y_test = get_data(uhi_data, used_columns, split_ratio)
    print(X_train.shape)
    print(y_train.shape)
    sc = StandardScaler()
    sc_y = MinMaxScaler()
    X_train = sc.fit_transform(X_train)

    y_train = np.round(y_train, 5)
    y_train_processed = sc_y.fit_transform(y_train.reshape(-1, 1)).flatten()

    model.fit(X_train, y_train_processed)
    
    # Make predictions on the training data
    insample_predictions = model.predict(X_train)
    insample_predictions = sc_y.inverse_transform(insample_predictions.reshape(-1, 1)).flatten()
    # calculate R-squared score for in-sample predictions
    Y_train = y_train.tolist()
    r2_train = r2_score(Y_train, insample_predictions)
    if split_ratio == 0.0:
        return sc, sc_y, model, r2_train, 0.0
    
    X_test = sc.transform(X_test)
    # Make predictions on the test data
    outsample_predictions = model.predict(X_test)
    outsample_predictions = sc_y.inverse_transform(outsample_predictions.reshape(-1, 1)).flatten()
    # calculate R-squared score for out-sample predictions
    Y_test = y_test.tolist()
    r2_test = r2_score(Y_test, outsample_predictions)
    return sc, sc_y, model, r2_train, r2_test

# Combine two datasets vertically (along columns) using pandas concat function.
def combine_two_datasets(dataset1,dataset2):
    data = pd.concat([dataset1, dataset2], axis=1)
    return data

In [3]:
ground_df = pd.read_csv("Training_data_uhi_index.csv")
ground_df.head()


,Longitude,Latitude,datetime,UHI Index
0,-73.909167,40.813107,24-07-2021 15:53,1.030289
1,-73.909187,40.813045,24-07-2021 15:53,1.030289
2,-73.909215,40.812978,24-07-2021 15:53,1.023798
3,-73.909242,40.812908,24-07-2021 15:53,1.023798
4,-73.909257,40.812845,24-07-2021 15:53,1.021634


### (internal data) 1. Weather Data ###

#### based on the distance of the interested point to the location of the weather station, assign the weather data to the interested point


In [4]:
def define_area_and_weather(in_data):
    data = in_data.copy()
    Bronx_coords = (-73.89352,40.87248)
    Manhattan_coords = (-73.96449,40.76754)
    # Calculate L2 (Euclidean) distance between two points
    def l2_distance(point1, point2):
        return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

    # For each point in data, calculate distance to Bronx and Manhattan coordinates
    # and assign area based on which is closer
    data['area'] = data.apply(
        lambda row: 'Bronx' if l2_distance((row['Longitude'], row['Latitude']), Bronx_coords) < 
                            l2_distance((row['Longitude'], row['Latitude']), Manhattan_coords) 
                    else 'Manhattan', axis=1)

    manhattan_weather = [ 26.73333333,48.54166667  , 2.96666667, 167.91666667, 426.08333333]
    bronx_weather = [ 27.51666667 , 44.45833333 ,  3.125   ,   132.66666667, 454.91666667]

    # Add weather features based on area
    data['w1'] = data.apply(
        lambda row: manhattan_weather[0] if row['area'] == 'Manhattan' else bronx_weather[0], axis=1)
    data['w2'] = data.apply(
        lambda row: manhattan_weather[1] if row['area'] == 'Manhattan' else bronx_weather[1], axis=1)
    data['w3'] = data.apply(
        lambda row: manhattan_weather[2] if row['area'] == 'Manhattan' else bronx_weather[2], axis=1)
    data['w4'] = data.apply(
        lambda row: manhattan_weather[3] if row['area'] == 'Manhattan' else bronx_weather[3], axis=1)
    data['w5'] = data.apply(
        lambda row: manhattan_weather[4] if row['area'] == 'Manhattan' else bronx_weather[4], axis=1)
    data.drop(in_data.columns, axis=1, inplace=True)
    data.drop(columns=['area'], inplace=True)
    return data, ['w1','w2','w3','w4','w5']

weather_data, weather_columns = define_area_and_weather(ground_df)

###  (internal data) 2. BUILDING FOOTPRINT within various buffer sizes ###

for each interested point, we construct the buffer of it and caculate:
1. the number of buildings within the buffer [we consider buffer size in [0.06, 0.07, 0.08, 0.09, 0.1]]
2. the area of the building footprint within the buffer [we consider buffer size in [0.0005, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]]

the computation is done under projection: EPSG:4326






In [5]:
### NUMBER OF BUILDINGS ###
def get_number_buildings(lat, lon, buffer_size, gdf):
    point = Point(lon, lat)
    buffer = point.buffer(buffer_size)
    return len(gdf[gdf['geometry'].intersects(buffer)])

def process_row(index, row, buffer_sizes, gdf):
    result = {}
    for buffer_size in buffer_sizes:
        num_buildings = get_number_buildings(row['Latitude'], row['Longitude'], buffer_size, gdf)
        result[f'num_buildings_{buffer_size}'] = num_buildings
    return index, result

def construct_building_footprint(in_data, gdf, buffer_sizes):
    data = in_data.copy()
    import multiprocessing as mp

    pool = mp.Pool(mp.cpu_count())
    results = [pool.apply_async(process_row, args=(index, row, buffer_sizes, gdf)) for index, row in data.iterrows()]

    for res in tqdm(results):
        index, result = res.get()
        for key, value in result.items():
            data.at[index, key] = value

    pool.close()
    pool.join()
    data.drop(in_data.columns, axis=1, inplace=True)
    return data, ['num_buildings_{}'.format(buffer_size) for buffer_size in buffer_sizes]

### AREA OF BUILDING FOOTPRINT ###
def building_area_within_buffer(lat, lon, buffer_size, gdf):
    point = Point(lon, lat)
    buffer = point.buffer(buffer_size)
    overlap = gdf.intersection(buffer).area
    return (overlap).sum()

def process_row_area(index, row, buffer_sizes, gdf):
    result = {}
    for buffer_size in buffer_sizes:
        area = building_area_within_buffer(row['Latitude'], row['Longitude'], buffer_size, gdf)
        result[f'building_area_{buffer_size}'] = area
    return index, result

def construct_building_footprint_area(in_data, gdf, buffer_sizes):
    data = in_data.copy()
    import multiprocessing as mp

    pool = mp.Pool(mp.cpu_count())
    results = [pool.apply_async(process_row_area, args=(index, row, buffer_sizes, gdf)) for index, row in data.iterrows()]

    for res in tqdm(results):
        index, result = res.get()
        for key, value in result.items():
            data.at[index, key] = value

    pool.close()
    pool.join()
    data.drop(in_data.columns, axis=1, inplace=True)
    return data, [f'building_area_{buffer_size}' for buffer_size in buffer_sizes]



### AREA OF BUILDING FOOTPRINT ###
if not os.path.exists('cached_tables/train_building_footprint_area.csv'):

    kml_file = './raw_data/Building_Footprint.kml'
    building_geometry = gpd.read_file(kml_file, driver='KML')
    df = pd.read_csv('Training_data_uhi_index.csv')
    data, columns = construct_building_footprint_area(df, building_geometry, [0.0005,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1])
    data.to_csv('cached_tables/train_building_footprint_area.csv', index=False)

if not os.path.exists('cached_tables/val_building_footprint_area.csv'):
    df = pd.read_csv('Submission_template.csv')
    data, columns = construct_building_footprint_area(df, building_geometry, [0.0005,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1])
    data.to_csv('cached_tables/val_building_footprint_area.csv', index=False)

### NUMBER OF BUILDINGS ###
if not os.path.exists('cached_tables/train_building_footprint.csv'):

    kml_file = './raw_data/Building_Footprint.kml'
    building_geometry = gpd.read_file(kml_file, driver='KML')
    df = pd.read_csv('Training_data_uhi_index.csv')
    data, columns = construct_building_footprint(df, building_geometry, [0.06,0.07,0.08,0.09,0.1])
    data.to_csv('cached_tables/train_building_footprint.csv', index=False)

if not os.path.exists('cached_tables/val_building_footprint.csv'):
    df = pd.read_csv('Submission_template.csv')
    data, columns = construct_building_footprint(df, building_geometry, [0.06,0.07,0.08,0.09,0.1])
    data.to_csv('cached_tables/val_building_footprint.csv', index=False)

In [6]:
building_footprint_area = pd.read_csv('cached_tables/train_building_footprint_area.csv')
building_footprint_area_columns = ['building_area_0.0005', 'building_area_0.001', 'building_area_0.002', 'building_area_0.003', 'building_area_0.004', 'building_area_0.005', 'building_area_0.006', 'building_area_0.007', 'building_area_0.008', 'building_area_0.009', 'building_area_0.01', 'building_area_0.02', 'building_area_0.03', 'building_area_0.04', 'building_area_0.05', 'building_area_0.06', 'building_area_0.07', 'building_area_0.08', 'building_area_0.09', 'building_area_0.1' ]

building_footprint = pd.read_csv('cached_tables/train_building_footprint.csv')
building_footprint_columns = ['num_buildings_0.06', 'num_buildings_0.07', 'num_buildings_0.08', 'num_buildings_0.09', 'num_buildings_0.1' ]

###  (external data) 3. Building elevation within various buffer sizes ###

#### Data page: https://data.cityofnewyork.us/City-Government/Building-Elevation-and-Subgrade-BES-/bsin-59hv/about_data

#### Download: 
in terminal, run 
```bash
wget -O raw_data/nyc_building_elevation.csv  https://data.cityofnewyork.us/api/views/bsin-59hv/rows.csv
``` 

#### Data description:
1. z_grade: The elevation of the building at it's lowest adjacent grade - the lowest point where the building touches the ground (feet).
2. z_floor: The elevation of what is estimated to be the lowest actively used floor (feet)
3. the_geom: Geometry column used for mapping (will convert to geometry column later)

#### Feature derivation:
for each interested point, we construct the buffer of it and caculate:
1. average z_grade within the buffer
2. average z_floor within the buffer

we consider buffer size in [0.005, 0.01]

the computation is done under projection: EPSG:4326






In [7]:
def building_elevation_within_buffer(lat, lon, buffer_size, gdf):
    point = Point(lon, lat)
    buffer = point.buffer(buffer_size)
    z_grade = gdf[gdf.within(buffer)]['z_grade'].mean()
    z_floor = gdf[gdf.within(buffer)]['z_floor'].mean()
    
    return z_grade, z_floor

def process_row_elevation(index, row, buffer_sizes, gdf):
    result = {}
    for buffer_size in buffer_sizes:
        z_grade, z_floor = building_elevation_within_buffer(row['Latitude'], row['Longitude'], buffer_size, gdf)
        result[f'building_z_grade_{buffer_size}'] = z_grade
        result[f'building_z_floor_{buffer_size}'] = z_floor
    return index, result

def construct_building_elevation(in_data, gdf, buffer_sizes):
    data = in_data.copy()
    import multiprocessing as mp

    pool = mp.Pool(mp.cpu_count())
    results = [pool.apply_async(process_row_elevation, args=(index, row, buffer_sizes, gdf)) for index, row in data.iterrows()]

    for res in tqdm(results):
        index, result = res.get()
        for key, value in result.items():
            data.at[index, key] = value

    pool.close()
    pool.join()
    data.drop(in_data.columns, axis=1, inplace=True)
    return data, [f'building_z_grade_{buffer_size}' for buffer_size in buffer_sizes] + [f'building_z_floor_{buffer_size}' for buffer_size in buffer_sizes]

if not os.path.exists('cached_tables/train_building_elevation.csv'):
    # run this in terminal wget -O raw_data/nyc_building_elevation.csv  https://data.cityofnewyork.us/api/views/bsin-59hv/rows.csv
    elevation_file = './raw_data/nyc_building_elevation.csv'
    building_elevation = gpd.read_file(elevation_file)
    lower_left = (40.75, -74.01)
    upper_right = (40.88, -73.86)

    # cutoff the data to the area of interest
    mask1 = building_elevation['longitude'].astype(float) <= -73.86
    mask2 = building_elevation['longitude'].astype(float) >= -74.01
    mask3 = building_elevation['latitude'].astype(float) <= 40.88
    mask4 = building_elevation['latitude'].astype(float) >= 40.75
    mask = mask1 & mask2 & mask3 & mask4
    building_elevation = building_elevation[mask]
    building_elevation = building_elevation[['longitude','latitude','z_grade', 'z_floor']]
    building_elevation['geometry'] = gpd.points_from_xy(building_elevation['longitude'], building_elevation['latitude'])
    building_elevation = gpd.GeoDataFrame(building_elevation, geometry='geometry')
    building_elevation['z_grade'] = building_elevation['z_grade'].astype(float)
    building_elevation['z_floor'] = building_elevation['z_floor'].astype(float)

    df = pd.read_csv('Training_data_uhi_index.csv')
    data, columns = construct_building_elevation(df, building_elevation, [0.005, 0.01])
    data.to_csv('cached_tables/train_building_elevation.csv', index=False)

    df = pd.read_csv('Submission_template.csv')
    data, columns = construct_building_elevation(df, building_elevation, [0.005, 0.01])
    data.to_csv('cached_tables/val_building_elevation.csv', index=False)

In [8]:
building_elevation_data = pd.read_csv('cached_tables/train_building_elevation.csv')
building_elevation_columns = ['building_z_grade_0.005', 'building_z_floor_0.005', 'building_z_grade_0.01', 'building_z_floor_0.01']

###  (external data) 4. Building height within various buffer sizes ###

#### Data page: https://data.cityofnewyork.us/City-Government/Building-Footprints/5zhs-2jue/about_data

#### Download: 
in terminal, run 
```bash
wget -O raw_data/nyc_building_height.csv  https://data.cityofnewyork.us/api/views/5zhs-2jue/rows.csv
``` 

#### Data description:
1. HEIGHTROOF: The height of the roof above the ground elevation, not height above sea level.
2. BIN: Building Identification Number. A number assigned by City Planning and used by Dept. of Buildings to reference information pertaining to an individual building. The first digit is a borough code (1 = Manhattan, 2 = The Bronx, 3 = Brooklyn, 4 = Queens, 5 = Staten Island). The remaining 6 digits are unique for buildings within that borough. In some cases where these 6 digits are all zeros (e.g. 1000000, 2000000, etc.) the BIN is unassigned or unknown.
3. the_geom: Geometry column used for mapping (will convert to geometry column later)

#### Feature derivation:
for each interested point, we construct the buffer of it and caculate:
1. Floor Area Ratio (FAR): the measurement of a building's floor area in relation to the size of the lot/parcel

we consider buffer size in [500] meters

the computation is done under projection: EPSG:3857






In [9]:
def extract_building_far(gdf, location, buffer_size, standard_floor_height=3):
   
    if gdf.crs is None:
        raise ValueError("GeoDataFrame has no CRS, please set the coordinate reference system first.")
    
    gdf_projected = gdf.to_crs(epsg=3857)
    
    lon, lat = location
    point = Point(lon, lat)
    point_projected = gpd.GeoSeries([point], crs="EPSG:4326").to_crs(epsg=3857).iloc[0]
    
    buffer_polygon = point_projected.buffer(buffer_size)
    buffer_area = buffer_polygon.area 
    
    buildings_in_buffer = gdf_projected[gdf_projected.intersects(buffer_polygon)].copy()

    buildings_in_buffer['footprint_area'] = buildings_in_buffer.geometry.area
    
    # ------------------------------
    # 1. FAR computation
    buildings_in_buffer['estimated_floors'] = buildings_in_buffer['HEIGHTROOF'] / standard_floor_height
    buildings_in_buffer['floor_area'] = buildings_in_buffer['footprint_area'] * buildings_in_buffer['estimated_floors']
    total_floor_area = buildings_in_buffer['floor_area'].sum()
    far = total_floor_area / buffer_area if buffer_area > 0 else None
    
    features = {
        'FAR': far,
    }
    
    return features

def building_far_within_buffer(lat, lon, buffer_size, gdf):
    location = (lon, lat)
    result = extract_building_far(gdf, location, buffer_size)
    return result

def process_row_far(index, row, buffer_sizes, gdf):
    result = {}
    for buffer_size in buffer_sizes:
        result = building_far_within_buffer(row['Latitude'], row['Longitude'], buffer_size, gdf)
        result = {f'{k}@{buffer_size}':v for k,v in result.items()}
    return index, result

def construct_building_far(in_data, gdf, buffer_sizes):
    data = in_data.copy()
    import multiprocessing as mp

    pool = mp.Pool(16)
    results = [pool.apply_async(process_row_far, args=(index, row, buffer_sizes, gdf)) for index, row in data.iterrows()]

    for res in tqdm(results):
        index, result = res.get()
        for key, value in result.items():
            data.at[index, key] = value

    pool.close()
    pool.join()
    data.drop(in_data.columns, axis=1, inplace=True)
    return data, []
    
if not os.path.exists('cached_tables/train_building_far.csv'): 
    # https://data.cityofnewyork.us/api/views/5zhs-2jue/rows.csv
    building_height_file = './raw_data/nyc_building_height.csv'
    building_height = gpd.read_file(building_height_file)
    lower_left = (-74.01, 40.75)
    upper_right = (-73.86, 40.88)

    building_height['geometry'] = building_height['the_geom'].apply(wkt.loads)
    building_height = gpd.GeoDataFrame(building_height, geometry='geometry')

    building_height = building_height[['geometry', 'HEIGHTROOF']]
    building_height['HEIGHTROOF'] = building_height['HEIGHTROOF'].astype(float)
    building_height = gpd.GeoDataFrame(building_height, geometry='geometry')


    tile_polygon = gpd.GeoDataFrame(geometry=[Polygon([lower_left, (lower_left[0], upper_right[1]), upper_right, (upper_right[0], lower_left[1]), lower_left])])
    building_height = gpd.clip(building_height, tile_polygon)
    building_height.set_crs(epsg=4326, inplace=True)

    mask1 = building_height.to_crs(epsg=3857).geometry.area>50
    mask2 = building_height['HEIGHTROOF'] > 2
    mask = mask1 & mask2
    building_height = building_height[mask]

    df = pd.read_csv('Training_data_uhi_index.csv')
    data, columns = construct_building_far(df, building_height, [500])
    data.to_csv('data/train_building_far.csv', index=False)

    df = pd.read_csv('Submission_template.csv')
    data, columns = construct_building_far(df, building_height, [500]) # 500,
    data.to_csv('data/val_building_far.csv', index=False)

In [10]:
building_far_data = pd.read_csv('cached_tables/train_building_far.csv')
building_far_columns = ['FAR@500']

###  (external data) 5. Building energy consumption within various buffer sizes ###

#### Data page: https://energy.cusp.nyu.edu/#/

#### Download: 
in terminal, run 
```bash
wget -O raw_data/nyc_building_energy.csv 'https://uil.carto.com/api/v2/sql?format=csv&filename=filtered_evt&q=SELECT%20*%20FROM%20table_2021_disclosure_ll84_viz_11_2022%20WHERE%20(year%20%3E=%201900%20AND%20year%20%3C=%202022)%20AND%20(gfa%20%3E=%2010000%20AND%20gfa%20%3C=%20500035)%20%20AND%20(eui%20%3E=%2050%20AND%20eui%20%3C=%20350)'
``` 

#### Data description:
1. Weather Normalized Source Energy Use Intensity (EUI): Total amount of the energy from all the raw fuel required to operate a property, including losses that take place during generation, transmission, and distribution normalized by the building gross square footage and expressed in kBtu per gross square foot (kBtu/ft2) of the property normalized for weather. Weather normalization facilitates comparison between different parts of the country and corrects for year-to-year differences in weather. Weather Normalized Source EUI values are the result of self-reported entries
2. Water Use Intensity (WUI): The annual consumption of water in gallons per gross square foot (gal/ft2) of the property.
3. Greenhouse Gas Intensity (GHG): The total direct and indirect greenhouse gases emitted due to energy used by the property per gross square foot of the property, reported in kilograms of carbon dioxide equivalent per square foot (kgCO2e/ft2). The carbon coefficient is based on New York City's EPA Emissions & Generation Resource Integrated Database (eGRID) sub-region.
4. Borough, Block, Lot Number (BBL): It is a 10-digit identifier assigned by NYC Department of Finance for each property in New York City and described by a set of 3 numbers: borough code followed by the tax block and the tax lot. BBLs are used by many city agencies to identify real estate for taxes, zoning, construction, and other purposes.
5. Gross Floor Area: Gross square footage of the property, per Department of Finance records.
6. ENERGY STAR® Score: A 1-to-100 percentile ranking for specified building types, as calculated by Portfolio Manager, with 100 being the best score and 50 the median. It compares the energy performance of a building against the national Commercial Buildings Energy Consumption Survey (CBECS) database, and independent industry surveys for that building type. This rating is normalized for weather and building attributes in order to obtain a measure of efficiency.
7. Building Energy Efficiency Rating: Building Energy Efficiency Rating is an A-D letter grade and a corresponding label based on the ENERGY STAR® Score that are required to be displayed near the entrance of certain buildings over 25,000 square feet in an effort to publicly disclose energy benchmarking information and to give New Yorkers a snapshot of the building's energy performance relative to its peers.
8. Property Type: The self-reported property type, as selected from the property type options available in Portfolio Manager. This is not necessarily consistent with the property type designation in Department of Finance records. The building types listed on the main page are the top 14 categories with Other containing all remaining categories reported in the disclosure data that meet the criteria.


#### Data preprocessing:
We have the BIN (Building Identification Number) in each row but it does not come with the geometry. Thus we utilize the building_height data to merge the geometry to the building_energy data, since the building_height data has the BIN column.

#### Feature derivation:
for each interested point, we construct the buffer of it and caculate:
1. EUI: the average EUI within the buffer

we consider buffer size in [0.005, 0.01, 0.15], we found other columns are not useful for our model

the computation is done under projection: EPSG:4326


In [11]:

def building_energy_within_buffer(lat, lon, buffer_size, gdf):
    point = Point(lon, lat)
    buffer = point.buffer(buffer_size)
    mask = gdf.intersects(buffer)
    eui = gdf[mask]['eui'].mean()
    return eui

def process_row_energy(index, row, buffer_sizes, gdf):
    result = {}
    for buffer_size in buffer_sizes:
        eui = building_energy_within_buffer(row['Latitude'], row['Longitude'], buffer_size, gdf)
        result[f'building_eui_{buffer_size}'] = eui
    return index, result

def construct_building_energy(in_data, gdf, buffer_sizes):
    data = in_data.copy()
    import multiprocessing as mp

    pool = mp.Pool(mp.cpu_count())
    results = [pool.apply_async(process_row_energy, args=(index, row, buffer_sizes, gdf)) for index, row in data.iterrows()]

    for res in tqdm(results):
        index, result = res.get()
        for key, value in result.items():
            data.at[index, key] = value

    pool.close()
    pool.join()
    data.drop(in_data.columns, axis=1, inplace=True)
    return data, [f'building_energy_{buffer_size}' for buffer_size in buffer_sizes]


if not os.path.exists('cached_tables/train_building_energy.csv'):

    building_energy = pd.read_csv('./raw_data/nyc_building_energy.csv')

    building_height_file = './raw_data/nyc_building_height.csv'
    building_height = gpd.read_file(building_height_file)
    building_height['geometry'] = building_height['the_geom'].apply(wkt.loads)
    building_height = gpd.GeoDataFrame(building_height, geometry='geometry')
    building_height = building_height[['geometry', 'BIN']]
    building_height['bin'] = building_height['BIN']

    building_energy = building_energy.merge(building_height, on='bin', how='left')
    building_energy = building_energy[['geometry','energy', 'eui', 'wui', 'ess', 'gfa', 'ghg', 'value']]
    building_energy.dropna(inplace=True)
    building_energy = gpd.GeoDataFrame(building_energy, geometry='geometry')

    df = pd.read_csv('Training_data_uhi_index.csv')
    data, columns = construct_building_energy(df, building_energy, [0.005,0.01,0.15])
    data.to_csv('cached_tables/train_building_energy.csv', index=False)

    df = pd.read_csv('Submission_template.csv')
    data, columns = construct_building_energy(df, building_energy, [0.005,0.01,0.15])
    data.to_csv('cached_tables/val_building_energy.csv', index=False)
    

In [12]:
building_energy_data = pd.read_csv('cached_tables/train_building_energy.csv')
building_energy_columns = ['building_eui_0.005', 'building_eui_0.01', 'building_eui_0.15']

###  (external data) 6. Population data in NYC ###

#### Data page: https://www.kaggle.com/datasets/muonneutrino/new-york-city-census-data

#### Download: 
You need to download the data from the link above and put it in the raw_data folder, there are two files:
1. nyc_census_tracts.csv
2. census_block_loc.csv

#### Data description of nyc_census_tracts.csv:
1. CensusTract: Census tract code
2. TotalPop: Population of the census tract

#### Data description of census_block_loc.csv:
1. BlockCode: Block code
2. Latitude: Latitude of the block
3. Longitude: Longitude of the block

#### Data preprocessing:
Note that BlockCode = '{CensusTract}{region id}', we we can merge the two table by merging on CensusTract and prefixing BlockCode. The final table has the following columns of interest:
1. TotalPop: Population of the census tract
2. Latitude: Latitude of the block
3. Longitude: Longitude of the block

There are other statistics in the table, which are not used in this project.

#### Feature derivation:
for each interested point, we find the cloese data point in the population and assign the TotalPop to the interested point:
1. TotalPop

the computation is done under projection: EPSG:4326


In [13]:

def find_closest_location(block_loc, row, coords):
    ground_location = np.array([row['Latitude'], row['Longitude']])
    # print(coords.shape)
    distances = ((coords - ground_location)**2).mean(axis=1)
    closest_index = np.argmin(distances)
    return block_loc.iloc[closest_index]

if not os.path.exists('cached_tables/train_pop_data.csv'):

    block_loc = pd.read_csv('raw_data/census_block_loc.csv')
    pop_nyc = pd.read_csv('raw_data/nyc_census_tracts.csv')

    block_loc['BlockCode'] = block_loc['BlockCode'].astype(str).str[:11]

    #rename block code to census tract
    block_loc.rename(columns={'BlockCode': 'CensusTract'}, inplace=True)
    pop_nyc['CensusTract'] = pop_nyc['CensusTract'].astype(str)

    #merge  block_loc and pop_nyc by CensusTract
    block_loc = pd.merge(block_loc, pop_nyc, on='CensusTract', how='left')
    block_loc.drop(columns=['County_x','State','County_y','Borough','CensusTract'], inplace=True)

    block_loc = block_loc[block_loc['TotalPop'].notna()]
    block_loc = block_loc[block_loc['TotalPop']!=0]

    block_loc.to_csv('raw_data/nyc_pop.csv', index=False)

    lats = block_loc['Latitude'].astype(float)
    longs = block_loc['Longitude'].astype(float)

    lats = np.array(lats)
    longs = np.array(longs)
    coords = np.column_stack((lats, longs))

    data = pd.read_csv('Training_data_uhi_index.csv')
    data = data.apply(lambda row: find_closest_location(block_loc, row, coords), axis=1)
    data.to_csv('cached_tables/train_pop_data.csv', index=False)


    data = pd.read_csv('Submission_template.csv')
    data = data.apply(lambda row: find_closest_location(block_loc, row, coords), axis=1)
    data.to_csv('cached_tables/val_pop_data.csv', index=False)


In [14]:
pop_data = pd.read_csv('cached_tables/train_pop_data.csv')
pop_columns = ['TotalPop']

### (external data) 7. Mapping Inequality Data in NYC ###

#### Data page: https://dsl.richmond.edu/panorama/redlining/data/NY-Manhattan/ and https://dsl.richmond.edu/panorama/redlining/data/NY-Bronx

#### Download: 
in terminal, run 
```bash
wget -O raw_data/bronx.json  'https://dsl.richmond.edu/panorama/redlining/static/citiesData/NYBronx1938/geojson.json'
wget -O raw_data/manhattan.json  'https://dsl.richmond.edu/panorama/redlining/static/citiesData/NYManhattan1937/geojson.json'
``` 

#### Data description
1. category_id: measure the income grade of the region
2. area: the area of the region
3. residential: category of the region
4. commercial: category of the region
5. industrial: category of the region

#### Feature derivation:
for each interested point, we find the region that the point is in and assign the following features to the interested point:
1. category_id: is consider ordinal data
2. area: is consider numerical data
3. residential: is consider binary data
4. commercial: is consider binary data
5. industrial: is consider binary data

the computation is done under projection: EPSG:4326


In [15]:
if not os.path.exists('cached_tables/train_inequality.csv'):

    bronx = gpd.read_file('raw_data/bronx.json')
    manhattan = gpd.read_file('raw_data/manhattan.json')

    ineq_df = pd.concat([bronx, manhattan], ignore_index=True)
    ineq_df = ineq_df[['geometry','category_id','area','residential','commercial','industrial']]
    ineq_df['residential'] = ineq_df['residential'].astype(int)
    ineq_df['commercial'] = ineq_df['commercial'].astype(int)
    ineq_df['industrial'] = ineq_df['industrial'].astype(int)
    ineq_df.loc[ineq_df['category_id'] == 276, 'category_id'] = 5
    ineq_df.loc[ineq_df['category_id'] == 202, 'category_id'] = 6
    ineq_df.loc[ineq_df['category_id'] == 16, 'category_id'] = 7
    ineq_df.loc[ineq_df['category_id'] == 205, 'category_id'] = 8


    def get_category_id(row):
        matching_categories = ineq_df[ineq_df.geometry.contains(Point(row['Longitude'], row['Latitude']))]['category_id'].values
        return matching_categories[0] if len(matching_categories) > 0 else None

    def get_area(row):
        matching_categories = ineq_df[ineq_df.geometry.contains(Point(row['Longitude'], row['Latitude']))]['area'].values
        return matching_categories[0] if len(matching_categories) > 0 else None

    def get_residential(row):
        matching_categories = ineq_df[ineq_df.geometry.contains(Point(row['Longitude'], row['Latitude']))]['residential'].values
        return matching_categories[0] if len(matching_categories) > 0 else None

    def get_commercial(row):
        matching_categories = ineq_df[ineq_df.geometry.contains(Point(row['Longitude'], row['Latitude']))]['commercial'].values
        return matching_categories[0] if len(matching_categories) > 0 else None

    def get_industrial(row):
        matching_categories = ineq_df[ineq_df.geometry.contains(Point(row['Longitude'], row['Latitude']))]['industrial'].values
        return matching_categories[0] if len(matching_categories) > 0 else None

    data = pd.read_csv('Training_data_uhi_index.csv')
    data['area'] = data.apply(get_area, axis=1)
    data['residential'] = data.apply(get_residential, axis=1)
    data['commercial'] = data.apply(get_commercial, axis=1)
    data['industrial'] = data.apply(get_industrial, axis=1)
    data['category_id'] = data.apply(get_category_id, axis=1)
    data['category_id'].fillna(9, inplace=True)
    data['area'].fillna(data['area'].mean(),inplace=True)
    data['residential'].fillna(1, inplace=True)
    data['commercial'].fillna(0, inplace=True)
    data['industrial'].fillna(0, inplace=True)
    data = data[['category_id','area','residential','commercial','industrial']]
    data.to_csv('cached_tables/train_inequality.csv',index=False)


    data = pd.read_csv('Submission_template.csv')
    data['category_id'] = data.apply(get_category_id, axis=1)
    data['area'] = data.apply(get_area, axis=1)
    data['residential'] = data.apply(get_residential, axis=1)
    data['commercial'] = data.apply(get_commercial, axis=1)
    data['industrial'] = data.apply(get_industrial, axis=1)
    data['category_id'].fillna(9, inplace=True)
    data['area'].fillna(data['area'].mean(),inplace=True)
    data['residential'].fillna(1,inplace=True)
    data['commercial'].fillna(0, inplace=True)
    data['industrial'].fillna(0, inplace=True)
    data = data[['category_id','area','residential','commercial','industrial']]
    data.to_csv('cached_tables/val_inequality.csv',index=False)

In [16]:
inequality_data = pd.read_csv('cached_tables/train_inequality.csv')
inequality_columns = ['category_id','area','residential','commercial','industrial']

### (external data) 8. Commute Data in NYC ###

#### Data page: https://a816-dohbesp.nyc.gov/IndicatorPublic/data-explorer/walking-driving-and-cycling/?id=2415#display=summary

#### Download: 
In https://a816-dohbesp.nyc.gov/IndicatorPublic/data-explorer/walking-driving-and-cycling/?id=2415#display=summary, download the data by clicking the "Download Data" button. Save it as raw_data/nyc_commute.csv.

In https://github.com/nychealth/EHDP-data/blob/production/geography/GeoLookup.csv, download the table and save it as raw_data/GeoLookup.csv.

#### Data description of nyc_commute.csv:
We have 'Bicycle (number)','Car, truck, or van (number)','Public transportation (number)','Walked (number)', 'Bicycle (percent)', 'Car, truck, or van (percent)', 'Public transportation (percent)', 'Walked (percent)' for each GeoID in NYC


#### Data description of GeoLookup.csv:
Provides the longitude and latitude of each GeoID in nyc_commute.csv

#### Feature derivation:
for each interested point, we find the GeoID that the point is cloest to and assign the following features to the interested point:
1. Bicycle (number)
2. Car, truck, or van (number)
3. Public transportation (number)
4. Walked (number)
5. Bicycle (percent)
6. Car, truck, or van (percent)
7. Public transportation (percent)
8. Walked (percent)

the computation is done under projection: EPSG:4326

In [17]:
if not os.path.exists('cached_tables/train_commute.csv'):

    commute = gpd.read_file('raw_data/nyc_commute.csv')
    commute = commute[commute['TimePeriod']=='2017-21']
    geolookup = pd.read_csv('raw_data/GeoLookup.csv')

    geolookup['GeoID'] = geolookup['GeoID'].astype(int)
    commute['GeoID'] = commute['GeoID'].astype(int)
    commute_geoid = commute.merge(geolookup, on='GeoID', how='left')
    commute_geoid = commute_geoid[['Lat','Long','Bicycle (number)','Car, truck, or van (number)','Public transportation (number)','Walked (number)', 
                                'Bicycle (percent)', 'Car, truck, or van (percent)', 'Public transportation (percent)', 'Walked (percent)']]

    commute_geoid['Long'] = commute_geoid['Long'].astype(float)
    commute_geoid['Lat'] = commute_geoid['Lat'].astype(float)
    commute_geoid['Bicycle (number)'] = commute_geoid['Bicycle (number)'].map(lambda x: int(x.replace(',', '')) if x is not None else None)
    commute_geoid['Car, truck, or van (number)'] = commute_geoid['Car, truck, or van (number)'].map(lambda x: int(x.replace(',', '')) if x is not None else None)
    commute_geoid['Public transportation (number)'] = commute_geoid['Public transportation (number)'].map(lambda x: int(x.replace(',', '')) if x is not None else None)
    commute_geoid['Walked (number)'] = commute_geoid['Walked (number)'].map(lambda x: int(x.replace(',', '')) if x is not None else None)
    commute_geoid['Bicycle (percent)'] = commute_geoid['Bicycle (percent)'].map(lambda x: float(x.replace('%', '')) if x is not None else None)
    commute_geoid['Car, truck, or van (percent)'] = commute_geoid['Car, truck, or van (percent)'].map(lambda x: float(x.replace('%', '')) if x is not None else None)
    commute_geoid['Public transportation (percent)'] = commute_geoid['Public transportation (percent)'].map(lambda x: float(x.replace('%', '')) if x is not None else None)
    commute_geoid['Walked (percent)'] = commute_geoid['Walked (percent)'].map(lambda x: float(x.replace('%', '')) if x is not None else None)

    def get_bicycle(row):
        commute_geoid['distance'] = ((commute_geoid['Lat'] - row['Latitude'])**2 + (commute_geoid['Long'] - row['Longitude'])**2)**0.5
        matching_categories = commute_geoid.loc[commute_geoid['distance'].idxmin()]['Bicycle (number)']
        return matching_categories

    def get_car(row):
        commute_geoid['distance'] = ((commute_geoid['Lat'] - row['Latitude'])**2 + (commute_geoid['Long'] - row['Longitude'])**2)**0.5
        matching_categories = commute_geoid.loc[commute_geoid['distance'].idxmin()]['Car, truck, or van (number)']
        return matching_categories
    def get_public_transportation(row):
        commute_geoid['distance'] = ((commute_geoid['Lat'] - row['Latitude'])**2 + (commute_geoid['Long'] - row['Longitude'])**2)**0.5
        matching_categories = commute_geoid.loc[commute_geoid['distance'].idxmin()]['Public transportation (number)']
        return matching_categories

    def get_walked(row):
        commute_geoid['distance'] = ((commute_geoid['Lat'] - row['Latitude'])**2 + (commute_geoid['Long'] - row['Longitude'])**2)**0.5
        matching_categories = commute_geoid.loc[commute_geoid['distance'].idxmin()]['Walked (number)']
        return matching_categories

    def get_bicycle_percent(row):
        commute_geoid['distance'] = ((commute_geoid['Lat'] - row['Latitude'])**2 + (commute_geoid['Long'] - row['Longitude'])**2)**0.5
        matching_categories = commute_geoid.loc[commute_geoid['distance'].idxmin()]['Bicycle (percent)']
        return matching_categories

    def get_car_percent(row):
        commute_geoid['distance'] = ((commute_geoid['Lat'] - row['Latitude'])**2 + (commute_geoid['Long'] - row['Longitude'])**2)**0.5
        matching_categories = commute_geoid.loc[commute_geoid['distance'].idxmin()]['Car, truck, or van (percent)']
        return matching_categories

    def get_public_transportation_percent(row):
        commute_geoid['distance'] = ((commute_geoid['Lat'] - row['Latitude'])**2 + (commute_geoid['Long'] - row['Longitude'])**2)**0.5
        matching_categories = commute_geoid.loc[commute_geoid['distance'].idxmin()]['Public transportation (percent)']
        return matching_categories

    def get_walked_percent(row):    
        commute_geoid['distance'] = ((commute_geoid['Lat'] - row['Latitude'])**2 + (commute_geoid['Long'] - row['Longitude'])**2)**0.5
        matching_categories = commute_geoid.loc[commute_geoid['distance'].idxmin()]['Walked (percent)']
        return matching_categories


    data = pd.read_csv('Training_data_uhi_index.csv')
    data['bicycle'] = data.apply(get_bicycle, axis=1)
    data['car'] = data.apply(get_car, axis=1)
    data['public_transportation'] = data.apply(get_public_transportation, axis=1)
    data['walked'] = data.apply(get_walked, axis=1)
    data['bicycle_percent'] = data.apply(get_bicycle_percent, axis=1)
    data['car_percent'] = data.apply(get_car_percent, axis=1)
    data['public_transportation_percent'] = data.apply(get_public_transportation_percent, axis=1)
    data['walked_percent'] = data.apply(get_walked_percent, axis=1)
    data = data[['bicycle','car','public_transportation','walked', 'bicycle_percent', 'car_percent', 'public_transportation_percent', 'walked_percent']]
    data.to_csv('cached_tables/train_commute.csv',index=False)


    data = pd.read_csv('Submission_template.csv')
    data['bicycle'] = data.apply(get_bicycle, axis=1)
    data['car'] = data.apply(get_car, axis=1)
    data['public_transportation'] = data.apply(get_public_transportation, axis=1)
    data['walked'] = data.apply(get_walked, axis=1)
    data['bicycle_percent'] = data.apply(get_bicycle_percent, axis=1)
    data['car_percent'] = data.apply(get_car_percent, axis=1)
    data['public_transportation_percent'] = data.apply(get_public_transportation_percent, axis=1)
    data['walked_percent'] = data.apply(get_walked_percent, axis=1)
    data = data[['bicycle','car','public_transportation','walked', 'bicycle_percent', 'car_percent', 'public_transportation_percent', 'walked_percent']]
    data.to_csv('cached_tables/val_commute.csv',index=False)

In [18]:
commute_data = pd.read_csv('cached_tables/train_commute.csv')
commute_columns = ['public_transportation']

### (external data) 9.Air Quality in NYC ###

#### Data page: https://a816-dohbesp.nyc.gov/IndicatorPublic/data-explorer/air-quality/?id=2023#display=summary 

#### Download: 
In https://a816-dohbesp.nyc.gov/IndicatorPublic/data-explorer/air-quality/?id=2023#display=summary, download the data by clicking the "Download Data" button. Save it as raw_data/nyc_pm2.5.csv.

In https://github.com/nychealth/EHDP-data/blob/production/geography/GeoLookup.csv, download the table and save it as raw_data/GeoLookup.csv. (same as the GeoLookup.csv in the Commute Data section)

#### Data description of nyc_pm2.5.csv:
We have'10th percentile mcg/m3','90th percentile mcg/m3','Mean mcg/m3' for each GeoID in NYC


#### Data description of GeoLookup.csv:
Provides the longitude and latitude of each GeoID in nyc_pm2.5.csv

#### Feature derivation:
for each interested point, we find the GeoID that the point is cloest to and assign the following features to the interested point:
1. 10th percentile mcg/m3
2. 90th percentile mcg/m3
3. Mean mcg/m3

the computation is done under projection: EPSG:4326

In [19]:
if not os.path.exists('cached_tables/train_air_quality.csv'):

    pm25 = gpd.read_file('./raw_data/nyc_pm2.5.csv')
    pm25 = pm25[pm25['TimePeriod']=='Summer 2023']

    geolookup = pd.read_csv('./raw_data/GeoLookup.csv')

    pm25['GeoID'] = pm25['GeoID'].astype(int)

    pm25_geoid = pm25.merge(geolookup, on='GeoID', how='left')
    pm25_geoid = pm25_geoid[['Lat','Long','10th percentile mcg/m3','90th percentile mcg/m3','Mean mcg/m3']]
    pm25_geoid.rename(columns={'10th percentile mcg/m3':'10th pm25','90th percentile mcg/m3':'90th pm25','Mean mcg/m3':'Mean pm25'}, inplace=True)
    pm25_geoid.dropna(inplace=True)

    def get_pm25(row):
        pm25_geoid['distance'] = ((pm25_geoid['Lat'] - row['Latitude'])**2 + (pm25_geoid['Long'] - row['Longitude'])**2)**0.5
        matching_categories = pm25_geoid.loc[pm25_geoid['distance'].idxmin()]['Mean pm25']
        return matching_categories

    def get_pm25_10th(row):
        pm25_geoid['distance'] = ((pm25_geoid['Lat'] - row['Latitude'])**2 + (pm25_geoid['Long'] - row['Longitude'])**2)**0.5
        matching_categories = pm25_geoid.loc[pm25_geoid['distance'].idxmin()]['10th pm25']
        return matching_categories


    data = pd.read_csv('Training_data_uhi_index.csv')

    data['pm25'] = data.apply(get_pm25, axis=1)
    data['pm25_10th'] = data.apply(get_pm25_10th, axis=1)

    data = data[['pm25','pm25_10th']]
    data.to_csv('cached_tables/train_air_quality.csv', index=False)


    data = pd.read_csv('Submission_template.csv')

    data['pm25'] = data.apply(get_pm25, axis=1)
    data['pm25_10th'] = data.apply(get_pm25_10th, axis=1)
    data = data[['pm25','pm25_10th']]
    data.to_csv('cached_tables/val_air_quality.csv', index=False)

In [20]:
air_quality_data = pd.read_csv('cached_tables/train_air_quality.csv')
air_quality_columns = ['pm25','pm25_10th']


### (external data) 10. Facilities in NYC ###

#### Data page: https://data.cityofnewyork.us/City-Government/Facilities-Database/ji82-xba5/about_data

#### Download: 
In https://data.cityofnewyork.us/City-Government/Facilities-Database/ji82-xba5/about_data, download the data by clicking the "Export" button. Save it as raw_data/nyc_facility.csv.

#### Data description of nyc_facility.csv:
The data contains the facilities in NYC, including the facility name, address, and the facility group. We only use the "PARKS AND PLAZAS" facility column

#### Feature derivation:
for each interested point, we build buffer with radius 0.01 and count the number of facilities in the buffer.
1. PARKS AND PLAZAS_count@0.01

the computation is done under projection: EPSG:4326

In [21]:
import pandas as pd
from tqdm import tqdm
if not os.path.exists('cached_tables/train_facility_features.csv'):

    facgroups = ['PARKS AND PLAZAS']
    def get_facility_data(lon, lat, df, radius=0.001):
        rows = df[abs(df['longitude'] - lon)<radius]
        rows = rows[abs(rows['latitude'] - lat)<radius]
        features = {}

        for facgroup in facgroups:
            facgroup_count = len([1 for fg in rows['facgroup'] if fg==facgroup])
            features[f'{facgroup}_count@{radius}'] = facgroup_count
    
        return rows, features

    facdb = pd.read_csv('./raw_data/nyc_facility.csv')
    facdb = facdb.drop(columns=['uid', 'addressnum', 'facname', 'streetname',	'address','bbl','cd','council','opabbrev','opname',	'city','bin','xcoord','ycoord',	'boro',	'borocode',	'zipcode','geometry'])


## train
    data = pd.read_csv('Training_data_uhi_index.csv')
    lons = data['Longitude'].values
    lats = data['Latitude'].values

    all_features = []

    for lon, lat in tqdm(zip(lons,lats),total=len(lons)):
        featuresall = {}
        for r in [0.01]:
            _, features = get_facility_data(lon, lat, facdb, radius=r)
            featuresall.update(features)
        all_features.append(featuresall)
    all_features = pd.DataFrame(all_features)
    all_features.to_csv('cached_tables/train_facility_features.csv', index=False)

### val
    data = pd.read_csv('Submission_template.csv')
    lons = data['Longitude'].values
    lats = data['Latitude'].values

    all_features = []
    for lon, lat in tqdm(zip(lons,lats),total=len(lons)):
        featuresall = {}
        for r in [0.01]:
            _, features = get_facility_data(lon, lat, facdb, radius=r)
            featuresall.update(features)
        all_features.append(featuresall)
    all_features = pd.DataFrame(all_features)
    all_features.to_csv('cached_tables/val_facility_features.csv', index=False)


In [22]:
facility_data = pd.read_csv('cached_tables/train_facility_features.csv')
facility_columns =  ['PARKS AND PLAZAS_count@0.01']

### Data merging, preprocessing ###

In [23]:
# 1.
uhi_data = combine_two_datasets(ground_df, weather_data)
# 2.
uhi_data = combine_two_datasets(uhi_data, building_footprint)
uhi_data = combine_two_datasets(uhi_data, building_footprint_area)
# 3.
uhi_data = combine_two_datasets(uhi_data, building_elevation_data)
# 4.
uhi_data = combine_two_datasets(uhi_data, building_far_data)
# 5.
uhi_data = combine_two_datasets(uhi_data, building_energy_data)
# 6.
uhi_data = combine_two_datasets(uhi_data, pop_data)
# 7.
uhi_data = combine_two_datasets(uhi_data, inequality_data)
# 8.
uhi_data = combine_two_datasets(uhi_data, commute_data)
# 9.
uhi_data = combine_two_datasets(uhi_data, air_quality_data)
# 10.
uhi_data = combine_two_datasets(uhi_data, facility_data)


In [24]:
columns_to_check =  weather_columns + building_footprint_columns + building_footprint_area_columns + building_elevation_columns + building_far_columns + building_energy_columns + pop_columns + inequality_columns + commute_columns + air_quality_columns + facility_columns
for col in columns_to_check:
    uhi_data[col] = uhi_data[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)
uhi_data.drop(columns=['Longitude','Latitude','datetime'], inplace=True)
mean_values = uhi_data.mean()
uhi_data = uhi_data.fillna(mean_values)

### Model fitting ###

In [25]:
from sklearn.ensemble import ExtraTreesRegressor
selected_columns = selected_columns = [
                        'w1',
                        'num_buildings_0.06', 
                        'num_buildings_0.07', 
                        'num_buildings_0.08',
                        'num_buildings_0.09', 
                        'num_buildings_0.1',
                        'building_area_0.0005',
                        'building_area_0.001',
                        'building_area_0.002',
                        'building_area_0.003',
                        'building_area_0.004',
                        'building_area_0.005',  
                        'building_area_0.006',
                        'building_area_0.007',
                        'building_area_0.008',
                        'building_area_0.009',
                        'building_area_0.01',
                        'building_area_0.02',
                        'building_area_0.03', 
                        'building_area_0.04', 
                        'building_area_0.05', 
                        'building_area_0.06', 
                        'building_area_0.07', 
                        'building_area_0.08',
                        'building_area_0.09',
                        'building_area_0.1',

                        'building_z_grade_0.005',
                        'building_z_floor_0.005',
                        'building_z_grade_0.01',
                        'building_z_floor_0.01',

                        'building_eui_0.005',
                        'building_eui_0.01',
                        'building_eui_0.15',
                        

                        'FAR@500',
                        'TotalPop', 

                        'category_id', 
                        'area', 
                        'residential' ,
                        'commercial', 
                        'industrial',
                        'public_transportation', 

                        'pm25',
                        'pm25_10th',
                        'PARKS AND PLAZAS_count@0.01',
                        'UHI Index']

model = ExtraTreesRegressor(n_estimators=5000, max_depth=None, n_jobs=-1, random_state=142, warm_start=True, max_features='sqrt', criterion='squared_error')

selected_columns_new = [c for c in selected_columns]
uhi_data = uhi_data[selected_columns]
columns = uhi_data.columns.tolist()
for col_1 in tqdm(columns):
    for col_2 in columns:
        if col_1 != 'UHI Index' and col_2 != 'UHI Index' and col_1 != col_2:
            uhi_data[f"{col_1}*{col_2}"] = uhi_data[col_1] * uhi_data[col_2]
            selected_columns_new.append(f"{col_1}*{col_2}")

uhi_data = uhi_data[selected_columns_new]
uhi_data.to_csv('train_crossed.csv', index=False)
# fit on full data
sc, sc_y, model, r2_train, r2_test = get_perf(uhi_data, model, selected_columns_new, split_ratio=0.)
print(f"r2_train: {r2_train}, r2_test: {r2_test}")


100%|██████████| 45/45 [00:01<00:00, 44.35it/s]


(11229, 1936)
(11229,)
r2_train: 1.0, r2_test: 0.0


### Model saving ###

In [27]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
    
# with open('model.pkl', 'rb') as f:
#     model = pickle.load(f)


### prediction ###

In [28]:
test_df = pd.read_csv('Submission_template.csv')
val_building_footprint = pd.read_csv('cached_tables/val_building_footprint.csv')
val_building_footprint_area = pd.read_csv('cached_tables/val_building_footprint_area.csv')
val_building_elevation_data = pd.read_csv('cached_tables/val_building_elevation.csv')
val_building_far_data = pd.read_csv('cached_tables/val_building_far.csv')
val_building_energy_data = pd.read_csv('cached_tables/val_building_energy.csv')
val_pop_data = pd.read_csv('cached_tables/val_pop_data.csv')
val_inequality_data = pd.read_csv('cached_tables/val_inequality.csv')
val_commute_data = pd.read_csv('cached_tables/val_commute.csv')
val_air_quality_data = pd.read_csv('cached_tables/val_air_quality.csv')
val_facility_data = pd.read_csv('cached_tables/val_facility_features.csv')

# 1.
val_data, _ = define_area_and_weather(test_df)
# 2.
val_data = combine_two_datasets(val_data, val_building_footprint)
val_data = combine_two_datasets(val_data, val_building_footprint_area)
# 3.
val_data = combine_two_datasets(val_data, val_building_elevation_data)
# 4.
val_data = combine_two_datasets(val_data, val_building_far_data)
# 5.
val_data = combine_two_datasets(val_data, val_building_energy_data)
# 6.
val_data = combine_two_datasets(val_data, val_pop_data)
# 7.
val_data = combine_two_datasets(val_data, val_inequality_data)
# 8.
val_data = combine_two_datasets(val_data, val_commute_data)
# 9.
val_data = combine_two_datasets(val_data, val_air_quality_data)
# 10.
val_data = combine_two_datasets(val_data, val_facility_data)


In [29]:
for col in columns_to_check:
    val_data[col] = val_data[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)
selected_columns.remove('UHI Index')
selected_columns_new.remove('UHI Index')
val_data = val_data[selected_columns]
val_data = val_data.fillna(mean_values)
columns = val_data.columns.tolist()
for col_1 in columns:
    for col_2 in columns:
        if col_1 != 'UHI Index' and col_2 != 'UHI Index' and col_1 != col_2:
            val_data[f"{col_1}*{col_2}"] = val_data[col_1] * val_data[col_2]

val_data.to_csv('val_crossed.csv', index=False)
transformed_val_data = sc.transform(val_data)

#Making predictions
final_predictions = model.predict(transformed_val_data)

final_predictions = sc_y.inverse_transform(final_predictions.reshape(-1, 1)).flatten()
final_prediction_series = pd.Series(final_predictions)
final_prediction_series = round(final_prediction_series, 5)

#Combining the results into dataframe
submission_df = pd.DataFrame({'Longitude':test_df['Longitude'].values, 'Latitude':test_df['Latitude'].values, 'UHI Index':final_prediction_series.values})

#Displaying the sample submission dataframe
submission_df.head()

submission_df.to_csv(f"Submission.csv",index = False)